# Exploring the Data

> Initial exploration of the data

For this project I am using a dataset about salaries for data related jobs (data engineering, data science, ML, DL, etc.). The raw data comes from ai-jobs.net. The dataset can be found [here](https://www.kaggle.com/datasets/ruchi798/data-science-job-salaries)

In [ ]:
#| default_exp explore_data

In [ ]:
#| hide
import warnings

In [ ]:
#| hide
warnings.filterwarnings("ignore", message="iteritems is deprecated")

In [ ]:
#| export
import kaggle
from pathlib import Path

In [ ]:
#| export
download_path = Path("../../tmp")
download_path.mkdir(exist_ok=True)
dataset_suffix = "ruchi798/data-science-job-salaries"

In [ ]:
#| hide
kaggle.api.dataset_download_files(dataset=dataset_suffix, path=download_path, unzip=True)

In [ ]:
#| hide
!ls ../../tmp

ds_salaries.csv


In [ ]:
#| export
import pandas as pd

In [ ]:
#| export
def download_data() -> pd.DataFrame:
    """Downloads the salaries dataset and reads it"""
    kaggle.api.dataset_download_files(dataset=dataset_suffix, path=download_path, unzip=True)
    return pd.read_csv(download_path / "ds_salaries.csv", index_col=0)

In [ ]:
data = download_data()
print(data.shape)
data.head()

(607, 11)


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


As we can see, the dataset consists of 607 position descriptions. The details for each can be found on the dataset [landing page](https://www.kaggle.com/datasets/ruchi798/data-science-job-salaries).

Right now, I just want to get used to using altair and to explore the dataset to see what concrete things I might want to find out and visualize, but, generally, the questions I want to answer are:

- How do employee's location and company's location affect salaries?
- How does experience affect salaries?
- How does company size affect salaries?
- What are the opportunities depending on if the work is remote, hybrid or on-site?

In [ ]:
import altair as alt

In [ ]:
alt.Chart(data).mark_circle().encode(
    x=alt.X(field="remote_ratio", type="quantitative", scale=alt.Scale(domain=[-10, 110])),
    y="salary_in_usd",
    color="experience_level",
    size="company_size",
    tooltip=["employment_type", "job_title", "employee_residence", "company_location"]
)

alt.Chart(...)

In [ ]:
data["company_in_us"] = (data["company_location"] == "US")
data["employee_in_us"] = (data["employee_residence"] == "US")

In [ ]:
alt.Chart(data).mark_circle().encode(
    x=alt.X(field="remote_ratio", type="quantitative", scale=alt.Scale(domain=[-10, 110])),
    y="salary_in_usd",
    color="company_in_us",
    tooltip=["employment_type", "job_title", "employee_residence", "company_location"]
)

alt.Chart(...)

In [ ]:
alt.Chart(data).mark_circle().encode(
    x=alt.X(field="remote_ratio", type="quantitative", scale=alt.Scale(domain=[-10, 110])),
    y="salary_in_usd",
    color="employee_in_us",
    tooltip=["employment_type", "job_title", "employee_residence", "company_location"]
)

alt.Chart(...)

In [ ]:
data["work_location"] = data["remote_ratio"].map({0: "On-site", 50: "Hybrid", 100: "Remote"})

In [ ]:
alt.Chart(data).mark_circle().encode(
    x="employment_type",
    y="salary_in_usd",
    color="work_location",
    tooltip=["job_title", "employee_residence", "company_location"]
).properties(width=400)

alt.Chart(...)

In [ ]:
data = data.astype({"work_year": str})

In [ ]:
alt.Chart(data).mark_circle().encode(
    x="work_year",
    y="salary_in_usd",
    color="company_in_us",
    tooltip=["job_title", "employee_residence", "company_location"]
).properties(width=400)

alt.Chart(...)

In [ ]:
alt.Chart(data).mark_bar().encode(
    y=alt.Y(field="salary_in_usd", type="quantitative", aggregate="max"),
    x=alt.X(field="company_location", type="nominal", sort=alt.EncodingSortField(field="salary_in_usd", op="max"))
)

alt.Chart(...)

In [ ]:
alt.Chart(data).mark_circle().encode(
    y=alt.Y(field="salary_in_usd", type="quantitative"),
    x=alt.X(field="company_location", type="nominal", sort=alt.EncodingSortField(field="salary_in_usd", op="max")),
    color="company_size",
    tooltip=["job_title", "employee_residence", "salary_in_usd"]
).properties(width=700)

alt.Chart(...)

In [ ]:
data["work_in_same_location"] = data["employee_residence"] == data["company_location"]

In [ ]:
alt.Chart(data).mark_circle().encode(
    y=alt.Y(field="salary_in_usd", type="quantitative"),
    x=alt.X(field="employee_residence", type="nominal", sort=alt.EncodingSortField(field="salary_in_usd", op="max")),
    color="work_in_same_location",
    tooltip=["job_title", "company_location", "company_size", "salary_in_usd"]
).properties(width=700)

alt.Chart(...)

In [ ]:
selection = alt.selection(type="multi", fields=["company_location"])
x_scale = alt.Scale(domain=[data.salary_in_usd.min(), data.salary_in_usd.max()])

scatter = alt.Chart(data).mark_circle().encode(
    x=alt.X(field="company_location", type="nominal", sort=alt.EncodingSortField(field="salary_in_usd", op="max")),
    y=alt.Y(field="salary_in_usd", type="quantitative"),
    color="company_size",
    tooltip=["job_title", "employee_residence", "salary_in_usd"],
    opacity=alt.condition(selection, alt.value(1),alt.value(.2))
).add_selection(selection).properties(width=450, height=250)

histogram = alt.Chart(data).mark_bar().encode(
    x=alt.X(field="salary_in_usd", type="quantitative", bin=alt.Bin(step=50000), scale=x_scale),
    y="count()",
    color="company_size",
    tooltip=["count()"]
).transform_filter(selection).properties(width=300, height=250).interactive()

scatter | histogram

alt.HConcatChart(...)

In [ ]:
selection = alt.selection(type="multi", fields=["company_size"])
x_scale = alt.Scale(domain=[data.salary_in_usd.min(), data.salary_in_usd.max()])

scatter = alt.Chart(data).mark_circle().encode(
    x=alt.X(field="company_size", type="nominal", sort=alt.EncodingSortField(field="salary_in_usd", op="max")),
    y=alt.Y(field="salary_in_usd", type="quantitative"),
    color="experience_level",
    tooltip=["job_title", "employee_residence", "salary_in_usd"],
    opacity=alt.condition(selection, alt.value(1),alt.value(.2))
).add_selection(selection).properties(width=450, height=250)

histogram = alt.Chart(data).mark_bar().encode(
    x=alt.X(field="salary_in_usd", type="quantitative", bin=alt.Bin(step=50000), scale=x_scale),
    y="count()",
    color="experience_level",
    tooltip=["count()"]
).transform_filter(selection).properties(width=300, height=250).interactive()

scatter | histogram

alt.HConcatChart(...)

## Observations

- The company location has a large effect on the salary
- Most people work in the same location as their residence, but if you are able to get a position in a foreign company you are highly likely to earn much more than in your home country, though, still less than if you were located in the same country
- Unsurprisingly, the more experience you have, the more you get paid, but there are a couple of things depending on the company size
  - If you have little or medium amount of experience, it doesn't really matter what the size of the company is
  - If you are senior, you will not earn as much if you are working for a small company
- For lower paying position all companies have a similar distribution of salaries, however, medium and large companies provide a large proportion of jobs. For the higher paying jobs, you almost have to work in middle to large sized companies
- Available opportunities for both remote and on-site seem to be roughly the same, but hybrid jobs might not have as many top paying ones
- Compared to 2020, there are more high paying jobs and fewer low paying ones


Now, with the key observations selected, we can focus on making good visualization for them. We are going to do that in the next notebook, but, for now, we can prepare the dataset for that.


In [ ]:
#| export

def prepare_dataset(data=None):
    if data is None:
        data = download_data()
    data["On-site/Remote"] = data["remote_ratio"].map({0: "On-site", 50: "Hybrid", 100: "Remote"})
    data["experience_level"] = data["experience_level"].map({
        "EN": "Entry-level / Junior",
        "MI": "Mid-level / Intermediate",
        "SE": "Senior-level / Expert",
        "EX": "Executive-level / Director"
    })
    data["employment_type"] = data["employment_type"].map({
        "PT": "Part-time",
        "FT": "Full-time",
        "CT": "Contract",
        "FL": "Freelance"
    })
    data["Number of Employees"] = data["company_size"].map({
        "S": "<50",
        "M": "50-250",
        "L": ">250"
    })
    data["Working for a Foreign Company"] = data["employee_residence"] != data["company_location"]
    data = data.astype({"work_year": str})
    data = data.drop(columns=["salary", "salary_currency", "remote_ratio", "company_size", "employment_type"])
    data = data.rename(columns={
        "work_year": "Work Year",
        "experience_level": "Experience Level",
#         "employment_type": "Employment Type",
        "job_title": "Job Title",
        "salary_in_usd": "Salary (usd)",
        "employee_residence": "Employee Residence",
        "company_location": "Company Location",
    })
    return data

In [ ]:
prepare_dataset().head()

,Work Year,Experience Level,Job Title,Salary (usd),Employee Residence,Company Location,On-site/Remote,Number of Employees,Working for a Foreign Company
0,2020,Mid-level / Intermediate,Data Scientist,79833,DE,DE,On-site,>250,False
1,2020,Senior-level / Expert,Machine Learning Scientist,260000,JP,JP,On-site,<50,False
2,2020,Senior-level / Expert,Big Data Engineer,109024,GB,GB,Hybrid,50-250,False
3,2020,Mid-level / Intermediate,Product Data Analyst,20000,HN,HN,On-site,<50,False
4,2020,Senior-level / Expert,Machine Learning Engineer,150000,US,US,Hybrid,>250,False
